In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import itertools
from tqdm import tqdm
import datetime
import time
import traceback

import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

import warnings
warnings.filterwarnings('ignore')

import simulation as sim
import metrics
import visualization as vis
import parser

# Одна итерация моделирования (для 1 структуры и 1 набора условий)

В примере считаем для всех структур:
1. доза = 1.0
2. фокус = 0.0
3. время сушки 90 с
4. время проявления 120 с

При необходимости, их можно вписать в параметры.

In [2]:
def iteration(intensity, 
              x_data,
              z_data,
              mode_peb,
              mode_develop,
              params):
    
    step_x = np.abs(x_data[1] - x_data[0])
    step_z = np.abs(z_data[1] - z_data[0])

    #Exposure
    my_exposure = sim.Exposure(params['dill_C'])
    my_m = my_exposure.simulate(intensity_in=intensity, dose=1.0)

    #PEB
    if mode_peb == 'noCAR':
        my_PEB_noCAR = sim.PEB_noCAR(sigma=np.sqrt(2 * params['da_peb'] * 90))
        my_peb = my_PEB_noCAR.simulate(step_x, step_z, m_concentration=my_m)
    elif mode_peb == 'CAR':
        my_PEB_CAR = sim.PEB_CAR(params['d_h'], params['d_q'], params['k_a'], params['k_q'], step_x, step_z)
        my_peb = my_PEB_CAR.simulate(params['q0'], params['w0'], 90, x_data, z_data, my_m)
    else:
        raise ValueError(f"Unknown mode: {mode_peb}")

    #Development rates
    if mode_develop == 'Mack':
        my_dev = sim.DevelopmentMack(params['rmin'], params['rmax'], params['n'],
                                     params['mth'])
        my_rates = my_dev.simulate(my_peb)
    elif mode_develop == 'Enhanced':
        my_dev = sim.DevelopmentEnhanced(params['rmin'], params['rmax'], 
                                         params['rresin'], params['n'], params['l'])
        my_rates = my_dev.simulate(my_peb)
    elif mode_develop == 'Notch':
        my_dev = sim.DevelopmentNotch(params['rmin'], params['rmax'], 
                                      params['n'], params['mth'], 
                                      params['nnotch'])
        my_rates = my_dev.simulate(my_peb)
    elif mode_develop == 'NotchEnhanced':
        my_dev = sim.DevelopmentNotchEnhanced(params['rmin'], params['rmax'],
                                              params['n'], params['mth'], 
                                              params['nnotch'], params['s'])
        my_rates = my_dev.simulate(my_peb)
    elif mode_develop == 'NotchInhibition':
        my_dev = sim.DevelopmentNotchInhibition(params['rmin'], params['rmax'], 
                                                params['n'], params['mth'], 
                                                params['nnotch'], params['delta'])
        my_rates = my_dev.simulate(my_peb, z_data)
    else:
        raise ValueError(f"Unknown mode: {mode_develop}")

    #Develop time contours
    my_timecont = sim.TimeContours()
    my_cont = my_timecont.simulate(x_data, z_data, my_rates)

    #Metrics
    my_metrics = metrics.Metrics(up_level=0.95, down_level=0.0, mode='poly3')
    left, right = my_metrics.find_contour(conts=my_cont, x_in=x_data, z_in=z_data, 
                                      x_stop_l=-300, x_stop_r=300, t=120)
    CD, rms_l, rms_l = my_metrics.find_metrics(left_points=left, right_points=right,
                                           z_data=z_data, CD_x=(0, 0), plot=False)   #CD_x не играет роли без графика
    
    return CD

# Считывание входных данных

Ввиду невозможности опубликования данных из реального исследования, распределения получены при достаточно случайных условиях и не несут физического смысла.

In [3]:
#название файла-примера входных данных 
filenames = [f"example_input/git_example_{i}.txt" for i in [200, 250, 280]]
CD_target = [200, 250, 280]  #пусть это будут целевые размеры
#в данном случае структура одномерная, поэтому распределение двумерное
data = []
for i in range(3):
    x_data, z_data, I_data = parser.parser_Optolithium(filenames[i])
    data.append({'I_data' : I_data[0], 'CD_target' : CD_target[i]})
#x_data, z_data одинаковые в нашем случае для всех 3х структур

# Подбор параметров (оптимизация перебором)

Будем перебирать параметры и сравнивать модели между собой по среднеквадратичному отклонению смоделированных размеров структур и target.

In [4]:
# Задаем параметры и их возможные значения
params={'dill_C': [1.1],
    'da_peb': [15.0],
    'q0': [None],
    'w0': [None],
    'd_h': [None],
    'd_q': [None],
    'k_a': [None],
    'k_q': [None],
    'rmin': [1, 0.5, 1.5],
    'rmax': [8, 9, 7],
    'rresin': [65, 67, 63],
    'n': [0, 1, 2, 3],
    'l': [8, 9, 10, 11, 12],
    'nnotch': [None],
    'mth': [None],
    's': [None],
    'delta': [None],
}

# Получаем все комбинации параметров
param_names = list(params.keys())
param_values = list(params.values())
parameters = []

for combination in itertools.product(*param_values):
    # Создаем словарь с текущей комбинацией параметров
    current_params = dict(zip(param_names, combination))
    parameters.append(current_params)

In [5]:
result = pd.DataFrame(columns=params.keys())

for elem in tqdm(parameters):
    p = elem.copy()
    CD_target = []
    CD_modeled = []
    for struct in data:
        I_data = struct['I_data']
        CD_target.append(struct['CD_target'])
        try:
            CD_tmp = iteration(I_data, x_data, z_data, 'noCAR', 'Enhanced', p)
            CD_modeled.append(CD_tmp)
        except Exception as exc:
            print(traceback.format_exc())
            CD_modeled.append(0)
    
    rms = np.sqrt(mean_squared_error(CD_target, CD_modeled))
    p['RMSerr'] = rms
    df_params = pd.DataFrame([p])
    result = pd.concat([result, df_params], ignore_index=True)

now = datetime.datetime.now().strftime("%Y-%m-%d__%H-%M")
#result.to_csv(f"result.csv")
print(f"Finished: {now}")
winsound.Beep(freq, duration)
result.sort_values(by='RMSerr').head(10)

100%|██████████| 540/540 [01:55<00:00,  4.68it/s]


Finished: 2025-04-01__14-43


,dill_C,da_peb,q0,w0,d_h,d_q,k_a,k_q,rmin,rmax,rresin,n,l,nnotch,mth,s,delta,RMSerr
162,1.1,15.0,None,None,None,None,None,None,1,7,63,0,10,None,None,None,None,1.492136
122,1.1,15.0,None,None,None,None,None,None,1,7,65,0,10,None,None,None,None,3.163760
202,1.1,15.0,None,None,None,None,None,None,0.5,8,67,0,10,None,None,None,None,4.757950
142,1.1,15.0,None,None,None,None,None,None,1,7,67,0,10,None,None,None,None,4.801401
324,1.1,15.0,None,None,None,None,None,None,0.5,7,67,0,12,None,None,None,None,4.947415
521,1.1,15.0,None,None,None,None,None,None,1.5,7,63,0,9,None,None,None,None,5.352269
304,1.1,15.0,None,None,None,None,None,None,0.5,7,65,0,12,None,None,None,None,6.337216
182,1.1,15.0,None,None,None,None,None,None,0.5,8,65,0,10,None,None,None,None,6.442366
40,1.1,15.0,None,None,None,None,None,None,1,8,63,0,8,None,None,None,None,6.854008
481,1.1,15.0,None,None,None,None,None,None,1.5,7,65,0,9,None,None,None,None,7.265899


Параметры, указанные в верхней строчке таблицы,считаем оптимальными (конечно же, из тех, что были перебраны).

**Внимание!** Полученные здесь результаты не несут никакого смысла, т.к.в качестве входных данных использовались достаточно случайные распределения. Все числа приведены здесь исключительно для примера.